In [ ]:
using Makie
using WGLMakie
using JSServe

In [ ]:
port = 7576

function setup_jsserve(proxy_base, port)
    JSServe.configure_server!(;listen_port = port, external_url = "$(proxy_base)/$(port)")
end

if "BINDER_SERVICE_HOST" in keys(ENV)
    url_snippet = js"""
    (function() {
        var loc = new URL(location);
        var bits = loc.pathname.split("/");
        loc.pathname = bits.slice(0, bits.indexOf("user") + 2).join("/") + "/proxy"
        return loc.href
    })()"""
else
    url_snippet = js"""new URL("/proxy", location.href).href"""
end
script = js"""
document.getElementById("paste_instructions").innerHTML = 
    "In the next cell paste the following:<br><pre>setup_jsserve(\\"" +
    $(url_snippet) +
    "\\", $(port))" """
JSServe.jsrender(Session(), DOM.div(id="paste_instructions", script))

In [ ]:
# Paste here

In [ ]:
fig = Figure()

ax = Axis(fig[1, 1])

toggles = [Toggle(fig, active = active) for active in [true, false]]
labels = [Label(fig, lift(x -> x ? "$l visible" : "$l invisible", t.active))
    for (t, l) in zip(toggles, ["sine", "cosine"])]

fig[1, 2] = grid!(hcat(toggles, labels), tellheight = false)

line1 = lines!(0..10, sin, color = :blue, visible = false)
line2 = lines!(0..10, cos, color = :red)

connect!(line1.visible, toggles[1].active)
connect!(line2.visible, toggles[2].active)

fig